<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
import pandas as pd
import folium

# Load the dataset from the uploaded file
file_path = '/Users/aishwarya/Downloads/spacex_launch_geo.csv'
spacex_df = pd.read_csv(file_path)

# Preview the data to confirm it loaded correctly
print(spacex_df.head())

# Extract latitude, longitude, and site name columns
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']]

# Create a map centered at the average latitude and longitude
center_lat = launch_sites['Lat'].mean()
center_long = launch_sites['Long'].mean()
launch_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Add markers for each launch site
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(launch_map)

# Save the map as an HTML file
launch_map.save("launch_sites_map.html")

# To display the map in a Jupyter Notebook
launch_map


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
from folium import DivIcon

# Create a map centered at the average latitude and longitude
launch_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Add circles and markers for each launch site
for _, site in launch_sites.iterrows():
    # Add a Circle at the launch site
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,  # Radius in meters
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(site['Launch Site'])).add_to(launch_map)
    
    # Add a Marker with a label at the launch site
    folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>',
        )
    ).add_to(launch_map)

# Save the map as an HTML file
launch_map.save("launch_sites_with_circles_and_markers.html")

# To display the map in a Jupyter Notebook
launch_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Define colors for outcomes
outcome_colors = {1: 'green', 0: 'red'}

# Add success and failure markers to the map
for _, launch in spacex_df.iterrows():
    folium.Circle(
        location=[launch['Lat'], launch['Long']],
        radius=50,  # Smaller radius for individual launches
        color=outcome_colors[launch['class']],  # Color based on success or failure
        fill=True,
        fill_opacity=0.6,
        popup=f"Site: {launch['Launch Site']}<br>Outcome: {'Success' if launch['class'] == 1 else 'Failure'}"
    ).add_to(launch_map)

# Save the enhanced map
launch_map.save("launch_sites_with_outcomes.html")

# Display the updated map in Jupyter Notebook
launch_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
import folium
from folium.plugins import MarkerCluster

# Create a map centered at the average latitude and longitude
site_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Define a function to determine marker color based on success or failure
def get_marker_color(outcome):
    return 'green' if outcome == 1 else 'red'

_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to current site_map
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
# Add a marker for each launch record
for _, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(
            color='white',
            icon_color=get_marker_color(record['class']),  # Success: Green, Failure: Red
            icon='ok-sign' if record['class'] == 1 else 'remove-sign'
        ),
        popup=f"Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Save the enhanced map with MarkerCluster
site_map.save("launch_sites_with_marker_cluster.html")

# Display the map in a Jupyter Notebook
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
from folium import DivIcon
from folium.plugins import MarkerCluster

# Create a map centered at the average latitude and longitude
launch_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Add circles and markers for each launch site
for _, site in launch_sites.iterrows():
    # Add a Circle at the launch site
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,  # Radius in meters
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(site['Launch Site'])).add_to(launch_map)
    
    # Add a Marker with a label at the launch site
    folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>',
        )
    ).add_to(launch_map)

# Save the map as an HTML file with site circles and labels
launch_map.save("launch_sites_with_circles_and_markers.html")

# Enhance the map with MarkerCluster for launch outcomes
# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(launch_map)

# Define a function to determine marker color based on success or failure
def get_marker_color(outcome):
    return 'green' if outcome == 1 else 'red'

# Add a marker for each launch record
for _, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(
            color='white',
            icon_color=get_marker_color(record['class']),  # Success: Green, Failure: Red
            icon='ok-sign' if record['class'] == 1 else 'remove-sign'
        ),
        popup=f"Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
launch_map.add_child(marker_cluster)

# Save the enhanced map with MarkerCluster
launch_map.save("launch_sites_with_marker_cluster.html")

# Display the map in a Jupyter Notebook
launch_map

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [14]:
import math
from folium import DivIcon
from geopy.distance import geodesic

# Define a function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Example launch site and closest coastline coordinates
launch_site_lat = 28.56342
launch_site_lon = -80.57674

# Selected closest coastline point (replace with actual coordinates after MousePosition interaction)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance between the launch site and the coastline
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add a marker for the closest coastline point
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>Coastline</b></div>',
    )
)
launch_map.add_child(coastline_marker)

# Add a marker displaying the distance between the launch site and coastline
distance_marker = folium.Marker(
    location=[(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_to_coastline:.2f} KM</b></div>',
    )
)
launch_map.add_child(distance_marker)

# Draw a PolyLine between the launch site and the coastline point
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color='blue'
)
launch_map.add_child(lines)

# Save the updated map
launch_map.save("launch_sites_with_distance_to_coastline.html")

# Display the map in Jupyter Notebook
launch_map

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
from folium import DivIcon
from geopy.distance import geodesic

# Example coordinates of the launch site
launch_site_lat = 28.56342
launch_site_lon = -80.57674

# Example coordinates of closest features (replace with actual coordinates)
closest_city_lat = 28.61283
closest_city_lon = -80.80765

closest_railway_lat = 28.5721
closest_railway_lon = -80.5851

closest_highway_lat = 28.56457
closest_highway_lon = -80.655

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Calculate distances
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, closest_railway_lat, closest_railway_lon)
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, closest_highway_lat, closest_highway_lon)

# Create a map centered at the launch site
launch_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add launch site marker
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>Launch Site</b></div>',
    )
).add_to(launch_map)

# Add markers for closest features with distances
for feature_name, feature_lat, feature_lon, distance in [
    ('Closest City', closest_city_lat, closest_city_lon, distance_to_city),
    ('Closest Railway', closest_railway_lat, closest_railway_lon, distance_to_railway),
    ('Closest Highway', closest_highway_lat, closest_highway_lon, distance_to_highway),
]:
    # Add feature marker
    folium.Marker(
        location=[feature_lat, feature_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#0000FF;"><b>{feature_name}</b></div>',
        )
    ).add_to(launch_map)

    # Add distance label marker
    folium.Marker(
        location=[(launch_site_lat + feature_lat) / 2, (launch_site_lon + feature_lon) / 2],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>',
        )
    ).add_to(launch_map)
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
    # Draw a PolyLine between the launch site and the feature
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [feature_lat, feature_lon]],
        weight=2,
        color='blue'
    ).add_to(launch_map)

# Display the map in Jupyter Notebook
launch_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
